In [1]:
import pandas as pd
f = open("day_12_input.txt")
input_list = f.read().split("\n")

In [2]:
commands_df = pd.DataFrame(input_list, columns=["command"])
commands_df['value'] = commands_df['command'].str[1:].astype(int)
commands_df['command'] = commands_df['command'].str[:1]
commands_df

,command,value
0,N,4
1,F,85
2,L,90
3,N,4
4,E,3
...,...,...
778,W,1
779,R,180
780,W,1
781,L,180


In [3]:
def day_12_part_1():

    #Calculate North/South and East/West movement. North, East are positive values.
    north_south_movement = commands_df.loc[commands_df['command'] == 'N', 'value'].sum() - commands_df.loc[commands_df['command'] == 'S', 'value'].sum()
    east_west_movement   = commands_df.loc[commands_df['command'] == 'E', 'value'].sum() - commands_df.loc[commands_df['command'] == 'W', 'value'].sum()
    #Create a mask for absolute movements and create a new dataframe that will contain only relative movements
    absolute_movement_mask = commands_df['command'].isin(list('NESW'))
    relative_commands_df = commands_df.loc[~absolute_movement_mask]

    #Delete "left" turns, these will now be "right".
    left_mask = relative_commands_df['command'] == 'L'
    relative_commands_df.loc[left_mask, 'command']  = 'R'
    relative_commands_df.loc[left_mask,   'value'] *= -1
    
    #Create a new key that will be used to combine values
    relative_commands_df['key'] = (relative_commands_df['command'] != relative_commands_df['command'].shift(1)).astype(int).cumsum()
    
    #Overwrite relative_commands with the groupby version
    relative_commands_df = relative_commands_df.groupby(['key', 'command']).sum()

    #Delete empty values and reset index
    relative_commands_df = relative_commands_df.loc[relative_commands_df['value'] != 0].reset_index()

    #Re-groupby
    relative_commands_df['key'] = (relative_commands_df['command'] != relative_commands_df['command'].shift(1)).astype(int).cumsum()
    relative_commands_df = relative_commands_df.groupby(['key', 'command']).sum()

    #Iterate on the resulting dataframe
    direction = 0
    for [key, command], row in relative_commands_df.iterrows():
        value = row['value']
        if command == 'F':
            if direction == 0:
                east_west_movement += value
            elif direction == 90:
                north_south_movement -= value
            elif direction == 180:
                east_west_movement -= value
            elif direction == 270:
                north_south_movement += value
        else:
            direction = (direction + value) % 360
    # return north_south_movement, east_west_movement
    return abs(north_south_movement) + abs(east_west_movement)

In [4]:
day_12_part_1()

1687

In [5]:
def day_12_part_2():
    #Start expressing all values as complex values
    waypoint_position = complex(10,1)
    ship_position = complex(0,0)
    #Dictionary containing the appropriate values for the turns. Note that 90 and 270 are flipped since we're doing "R" instead of "L"
    directions = {
        0:    complex( 1,  0),
        90:   complex( 0, -1),
        180:  complex(-1,  0),
        270:  complex( 0,  1),
    }
    #Create a copy of the commands_df, and turn NSW into complex numbers
    commands_df_copy = commands_df.copy()
    commands_df_copy.loc[commands_df_copy['command'] == 'N', 'value'] *= 1j
    commands_df_copy.loc[commands_df_copy['command'] == 'S', 'value'] *= -1j
    commands_df_copy.loc[commands_df_copy['command'] == 'W', 'value'] *= -1

    #Create a mask for the waypoint movements, then replace their command with an 'M'
    absolute_movement_mask = commands_df_copy['command'].isin(list('NESW'))
    commands_df_copy.loc[absolute_movement_mask, 'command'] = 'M'

    #As before, turn all 'left' turns into 'right' turns
    left_mask = commands_df_copy['command']  == 'L'
    commands_df_copy.loc[left_mask, 'command']  = 'R'
    commands_df_copy.loc[left_mask,   'value'] *= -1

    #Create a new key that will be used to combine values
    commands_df_copy['key'] = (commands_df_copy['command'] != commands_df_copy['command'].shift(1)).astype(int).cumsum()
    
    #Overwrite relative_commands with the groupby version
    commands_df_copy = commands_df_copy.groupby(['key', 'command']).sum()

    #Iterate over the resulting commands.
    for [key, command], row in commands_df_copy.iterrows():
        value = row['value']
        #If "M", move the waypoint
        if command == "M":
            waypoint_position += value
        #If "R", rotate by multiplying by the appropriate value
        elif command == 'R':
            mult = directions[int(value) % 360]
            waypoint_position *= mult
        #If "F", add the waypoint position the times indicated
        elif command == 'F':
            ship_position += waypoint_position * value
            
    #Return the sum of the absolute value of the real part and the imaginary part
    return int(abs(ship_position.real) + abs(ship_position.imag))

In [6]:
day_12_part_2()

20873